環境にTKET (pytket)と拡張機能(pytket-extension)をインストール。  
以下のセルを実行し、インストールしてください。

In [ ]:
!pip install -U pytket
!pip install -U pytket-qiskit
!pip install -U pytket-braket
!pip install -U pytket-qulacs
#!pip install -U pytket-qsharp #こちらのパッケージは.net等のインストールが必要です。
!pip install -U matplotlib #可視化のためのパッケージ
!pip install -U pylatexenc #可視化のためのパッケージ

pytket がインストールされたか確認

In [ ]:
!pip freeze |grep pytket

## 0. 今日の量子コンピュータの問題

- Noisy Intermediate-Scale Quantum (NISQ) デバイス
    - 量子回路が深くなる（ゲート数が多くなる）ほど、誤差が大きくなる
    - 十分な量子ビット数ではない
- 量子デバイスは特別なゲート演算のみが用意されている
- 特定のqubits間の量子ビット演算(multi qubit operation)しか用意されていない
- それぞれの量子デバイスに対して、量子ソフトウェアツールキットが用意されてる


### 0-1. TKETとは
- Quantum Software Development Kit
- TKETに実装されている回路最適化はC++で実装
- pythonモジュール　`pytket`で利用可能
- 最適化コンパイラ：　ユーザーフレンドリーな回路→量子デバイスで実行可能な回路に変換可能
    - Language-agnostic (多くの量子プログラミングフレームワーク(qiskit, Cirq, etc)をサポート)
    - Retagetable (多くの量子デバイス(IBM, Quantinuum, Amazon Braket(IonQ, Rigetti, OQC), Microsoft Azure Quantum(Quantinuu,IonQ, Rigetti) etc)をサポート)
    - Circuit Optimisation (量子計算時に生じるデバイスエラーの影響を最小化。デバイス依存＆デバイス非依存のものが実装)
    
<img src="./fig/tket1.png" width="750">



#### 参照
- [pytket ドキュメント](https://cqcl.github.io/tket/pytket/api/index.html)
- [pytket ユーザーマニュアル](https://cqcl.github.io/pytket/manual/index.html)
- [t|ket⟩ : A Retargetable Compiler for NISQ Devices](https://arxiv.org/abs/2003.10611)
- [TKET slack channel](https://join.slack.com/t/tketusers/shared_invite/zt-22295t975-eAsoPg0i4ANHajPAe61VzQ)

<img src="./fig/slack-qr.png" width="250">


### 0-2. 今回必要となる python パッケージ
Python Python 3.10.8で動作確認をしています。

|  パッケージ （version） |  概要  |
| :---- | :---- |
|  pytket  |  TKETを利用するためのpython モジュール  ( available for python3.9 or higher )|
|  pytket-qiskit  |  qiskitを利用するためのpytket-extension  |
|  pytket-braket  |  Amazon Braketを利用するためのpytket-extension  |
|  pytket-qsharp  |  Azure Quantumを利用するためのpytket-extension  |
|  pytket-circ    |  Google circを利用するためのpytket-extension  |
|  pytket-qulacs  |  Qulacsを利用するためのpytket-extension  |

<img src="./fig/tket2.png" width="750">

## 1. 量子回路を作成する
ここでは IBMの量子デバイスやシュミレーションを利用できる`qiskit`と`TKET`でGreenberger–Horne–Zeilinger状態を作成する。

### 1-1. `qiskit`で4 qubitsのGreenberger–Horne–Zeilinger状態を作成
$$ |\Psi\rangle = \frac{1}{\sqrt{2}}(|0000\rangle+|1111\rangle)$$

In [ ]:
from qiskit import QuantumCircuit

qs_ghz4 = QuantumCircuit(4)
qs_ghz4.h(0)
qs_ghz4.cx(0,1)
qs_ghz4.cx(0,2)
qs_ghz4.cx(0,3)
qs_ghz4.measure_all()

qs_ghz4.draw(output='mpl')

In [ ]:
#from qiskit.tools.visualization import circuit_drawer
#circuit_drawer(qs_ghz4, output='mpl')

### 1-2. IBMが提供しているローカルシミュレータで計算

In [ ]:
from qiskit import Aer
from qiskit.visualization import plot_histogram

In [ ]:
#Aer.backends()

In [ ]:
ibm_sim = Aer.get_backend('aer_simulator')

In [ ]:
handle = ibm_sim.run(qs_ghz4, shots=1000)
counts = handle.result().get_counts()
plot_histogram(counts)

### 1-3. `TKET`でGreenberger–Horne–Zeilinger状態を作成

In [ ]:
from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter

ghz4 = Circuit(4)
ghz4.H(0).CX(0,1).CX(0,2).CX(0,3)
ghz4.measure_all()
render_circuit_jupyter(ghz4)

### 1-4. `pytket-qiskit`でTKET 量子回路をIBMのローカルシミュレータで計算

In [ ]:
from pytket.extensions.qiskit import AerBackend
backend = AerBackend()

handle = backend.process_circuit(ghz4, n_shots =1000)
result_counts = backend.get_result(handle).get_counts()
plot_histogram(result_counts)

### 1-5. `pytket-qiskit`でTKET 量子回路をIBMの量子デバイスで計算

IBM tokenの設定

In [ ]:
path = 'key/ibm-token'
f = open(path)
ibm_token = f.read()
f.close()
#from pytket.extensions.qiskit.backends.config import set_ibmq_config
#set_ibmq_config(ibmq_api_token=ibm_token, instance=f"ibm-q/open/main")

In [ ]:
from qiskit_ibm_provider import IBMProvider
IBMProvider.save_account(token=ibm_token, overwrite=True)

In [ ]:
from pytket.extensions.qiskit import IBMQBackend, IBMQEmulatorBackend, AerBackend
from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter

In [ ]:
device = IBMQBackend.available_devices()

In [ ]:
[dev.device_name for dev in device]

##### ※注意 IBMから以下の発表があった
９月２６日周辺：以下のデバイスは引退を予定  
ibmq_lima, ibmq_belem, ibmq_quito, ibmq_manila, ibmq_jakarta.  
１０月２８日周辺：以下のデバイスは引退を予定  
ibm_perth, ibm_lagos, ibm_nairobi.

In [ ]:
# ibm_backend = AerBackend()
# ibm_backend = IBMQBackend("ibmq_lima")
# ibm_backend = IBMQBackend("ibmq_qasm_simulator")
# AerBackend to emulate the behaviour of IBMQBackend. Requires a valid IBMQ account.
ibm_backend = IBMQEmulatorBackend("ibmq_lima")

### IBMのクラウドシミュレータ（ibmq_lima）にジョブを実行
<img src="./fig/lima.png" width="250">  

IBM Quantum device `ibmq_lima` https://quantum-computing.ibm.com/services/resources?tab=systems&skip=20&system=ibmq_lima



In [ ]:
ibm_ghz4 = ibm_backend.get_compiled_circuit(ghz4)
render_circuit_jupyter(ibm_ghz4)
render_circuit_jupyter(ghz4)

In [ ]:
handle = ibm_backend.process_circuit(ibm_ghz4, n_shots =1000)
result_counts = ibm_backend.get_result(handle).get_counts()
plot_histogram(result_counts)

### 1-5. `pytket-qulacs`でTKET 量子回路をQulacsシミュレータで計算

In [ ]:
from pytket.extensions.qulacs import QulacsBackend
backend = QulacsBackend()

handle = backend.process_circuit(ghz4, n_shots =1000)
result_counts = backend.get_result(handle).get_counts()
plot_histogram(result_counts)

### GPU上でQulacsを利用している場合にも対応している


In [ ]:
#from pytket.extensions.qulacs import QulacsGPUBackend
#backend = QulacsGPUBackend()

#handle = backend.process_circuit(ghz4, n_shots =1000)
#result_counts = backend.get_result(handle).get_counts()
#plot_histogram(result_counts)

詳しくは
https://cqcl.github.io/pytket-qulacs/api/index.html
を参照ください

### 1-6. `pytket-braket`でTKET 量子回路をBraketシミュレータやBraketにある量子デバイスで計算

#### A. ローカル環境からBraketにアクセスされている方は、keyフォルダーのaws-keyのアクセスキー等の情報を入力し下記を実行してください

In [ ]:
#path = 'key/aws-key'
#f = open(path)
#aws_access_key_id, aws_secret_access_key, s3_name, bucket_key= [s.strip() for s in f.readlines()]
#f.close()

In [ ]:
#import boto3
#from braket.aws import AwsSession
#boto_session = boto3.Session(
#    aws_access_key_id= aws_access_key_id,
#    aws_secret_access_key= aws_secret_access_key,
#    region_name= 'us-east-1'
##    region_name= 'us-west-1'
##    region_name= 'eu-west-2'
#)
## us-east-1: IonQ, Simulators
## us-west-1: Rigetti, Simulators
## eu-west-2: Lucy, Simulators
##
#aws_session = AwsSession(boto_session=boto_session)

#### B. Sagemaker StudioからBraketにアクセスされている方は下記を実行してください。

In [ ]:
import boto3
from braket.aws import AwsSession
boto_session = boto3.Session(
    region_name= 'us-east-1'
#    region_name= 'us-west-1'
#    region_name= 'eu-west-2'
)
# us-east-1: IonQ, Simulators
# us-west-1: Rigetti, Simulators
# eu-west-2: Lucy, Simulators
#
aws_session = AwsSession(boto_session=boto_session)

<img src="./fig/s3_setting.png" width="750">

AWS環境に関する束野さん資料のP41から
https://jstouqsrh-my.sharepoint.com/:p:/g/personal/hironobu_noguchi_jstouqsrh_onmicrosoft_com/EV8r8PaW7i9EgockwNbf27IBdaS0oOnm

In [ ]:
s3_name = 'amazon-braket-osaka23'
bucket_key = 'user23-"your_number"/pytket'

In [ ]:
device = aws_session.search_devices()

In [ ]:
[dev for dev in device if dev["deviceStatus"]!='RETIRED']

In [ ]:
# backend の選択
from pytket.extensions.braket import BraketBackend

# ionQ   arn:aws:braket:us-east-1::device/qpu/ionq/Aria-1, region: us-east-1
# aws_backend = BraketBackend(device='Aria-1',region = 'us-east-1', s3_bucket=s3_name , s3_folder = bucket_key, device_type = 'qpu', provider = 'ionq', aws_session=aws_session)
# ionQ   arn:aws:braket:us-east-1::device/qpu/ionq/Harmony, region: us-east-1
# aws_backend = BraketBackend(device='Harmony',region = 'us-east-1', s3_bucket=s3_name , s3_folder = bucket_key, device_type = 'qpu', provider = 'ionq', aws_session=aws_session)
# Aspen-M-3   arn:aws:braket:us-west-1::device/qpu/rigetti/Aspen-M-1 #Aspen-M-1, region: us-west-1
# aws_backend = BraketBackend(device='Aspen-M-3',region = 'us-west-1', s3_bucket=s3_name , s3_folder = bucket_key, device_type = 'qpu', provider = 'rigetti', aws_session=aws_session)
# Oxford Lucy   arn:aws:braket:eu-west-2::device/qpu/oqc/Lucy #Lucy, region: eu-west-2
# aws_backend = BraketBackend(device='Lucy',region = 'eu-west-2', s3_bucket=s3_name , s3_folder = bucket_key, device_type = 'qpu', provider = 'oqc', aws_session=aws_session)
# SV1    sim_arn = 'arn:aws:braket:::device/quantum-simulator/amazon/sv1'     #SV1, region:
aws_backend = BraketBackend(device='sv1', s3_bucket=s3_name , s3_folder = bucket_key, aws_session=aws_session)
# TN1   sim_arn = 'arn:aws:braket:::device/quantum-simulator/amazon/tn1'    #TN1, region: us-west-2, us-east-1
# aws_backend = BraketBackend(device='tn1', s3_bucket=s3_name , s3_folder = bucket_key, device_type = 'quantum-simulator', provider = 'amazon', aws_session=aws_session)
# DM1   sim_arn = 'arn:aws:braket:::device/quantum-simulator/amazon/dm1'    #DM1, region: 
# aws_backend = BraketBackend(device='dm1', s3_bucket=s3_name , s3_folder = bucket_key, device_type = 'quantum-simulator', provider = 'amazon', aws_session=aws_session)
# Local sv1 simulator
# aws_backend = BraketBackend(local = True)

In [ ]:
aws_backend.backend_info

In [ ]:
from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter
# Greenberger–Horne–Zeilinger状態の作成
ghz4 = Circuit(4)
ghz4.H(0).CX(0,1).CX(0,2).CX(0,3)
ghz4.measure_all()
render_circuit_jupyter(ghz4)

# backend に用意されているゲートで量子回路を表現
aws_ghz4 = aws_backend.get_compiled_circuit(ghz4)
render_circuit_jupyter(aws_ghz4)

In [ ]:
aws_handle = aws_backend.process_circuit(aws_ghz4, n_shots =100)
aws_result = aws_backend.get_result(aws_handle)
aws_counts = aws_result.get_counts()
print(aws_counts)
from qiskit.visualization import plot_histogram
plot_histogram(aws_result.get_counts())

In [ ]:
aws_handle = aws_backend.process_circuit(ghz4, n_shots =100)
aws_result = aws_backend.get_result(aws_handle)
aws_counts = aws_result.get_counts()
print(aws_counts)
from qiskit.visualization import plot_histogram
plot_histogram(aws_result.get_counts())

### 1-7. `pytket-qsharp`でTKET 量子回路をMicrosoft Azure Quantumにある量子デバイスで計算が可能
本講演では詳細は割愛致しますが、Azure Quantum上でTKETをご利用になられたい方は下記のページ  
https://cqcl.github.io/pytket-qsharp/api/index.html  
または  
米澤(yasuyoshi.yonezawa@quantinuum.com)までご連絡ください。

## 2. 量子回路の最適化
量子計算時に生じるデバイスエラーの影響を最小化。
デバイス非依存のものが実装


### 2-1. `PauliSimp` 関数を利用した、量子回路の最適化
TKETには量子回路を最適化する様々な機能が用意されている。
ここでは、`PauliSimp` 関数を利用した回路の最適化を行う。

（`PauliSimp` 関数：Pauli ゲートとCliffordゲートで表現された量子回路を出力）

ランダムな量子回路を作成し、回路の深さとCXの深さを数える。

In [ ]:
from pytket.pauli import Pauli
from pytket.circuit import PauliExpBox, fresh_symbol, OpType
from pytket.passes import DecomposeBoxes
box = PauliExpBox([Pauli.I, Pauli.Z, Pauli.X, Pauli.Y], fresh_symbol('tm'))
from pytket.utils import Graph
import numpy as np

def get_random_pauli_gadgets(n_qubits, n_pauli_gadgets, max_entangle):
    """ """
    paulis = [Pauli.I, Pauli.X, Pauli.Y, Pauli.Z]
    circ = Circuit(n_qubits)
    for i in range(n_pauli_gadgets):
        ls_paulis = [np.random.choice(paulis) for k in range(max_entangle)]
        if ls_paulis.count(Pauli.Y) % 2 == 0:
            continue
        if len(ls_paulis) - ls_paulis.count(Pauli.I) <= 1:
            continue
        qubits = np.random.choice(
            [i for i in range(n_qubits)], size=max_entangle, replace=False
        )
        box = PauliExpBox(ls_paulis, fresh_symbol('a'))
        circ.add_pauliexpbox(box, sorted(qubits))
    DecomposeBoxes().apply(circ)
    return circ

circ = get_random_pauli_gadgets(
    n_qubits=8, n_pauli_gadgets=300, max_entangle=5
)
print('Circuit depth: ', circ.depth())
print('CX depth: ', circ.depth_by_type(OpType.CX))
render_circuit_jupyter(circ)

`PauliSimp` 関数を使って、量子回路の最適化

In [ ]:
# Circuit optimization by using compiler passes.
from pytket.passes import PauliSimp,FullPeepholeOptimise
circx = circ.copy()
PauliSimp().apply(circx)
#FullPeepholeOptimise().apply(circx)
print('Circuit depth: ', circx.depth())
print('CX depth: ', circx.depth_by_type(OpType.CX))
render_circuit_jupyter(circx)

## 3. 量子回路の変換
pytketでは
- qiskitで記述した量子回路(`qiskit.QuantumCircuit`)からTKETの量子回路のクラスに変換が可能
- TKETで記述した量子回路からqiskitの量子回路(`qiskit.QuantumCircuit`)のクラスに変換が可能
- TKETで記述した量子回路と他の量子プログラミング言語(一部)での量子回路の交換が可能

参照：[pytket-extensions](https://cqcl.github.io/pytket-extensions/api/index.html) 

### 3-1. `qiskit`の量子回路から`TKET`の量子回路に変換

In [ ]:
from pytket.extensions.qiskit import qiskit_to_tk

In [ ]:
ghz4_2 = qiskit_to_tk(qs_ghz4)
ghz4_2

In [ ]:
render_circuit_jupyter(ghz4_2)

### 3-2. `TKET`の量子回路から`qiskit`の量子回路に変換

In [ ]:
from pytket.extensions.qiskit import tk_to_qiskit

In [ ]:
qs_ghz4_2 = tk_to_qiskit(ghz4)
qs_ghz4_2

In [ ]:
qs_ghz4_2.draw('mpl')

### 3-3. `TKET`の量子回路から`braket`の量子回路に変換

In [ ]:
from pytket.extensions.braket import tk_to_braket
aws_ghz4_2 = tk_to_braket(ghz4)[0]
print(aws_ghz4_2)

### 3-4. `braket`の量子回路から`TKET`の量子回路に変換

In [ ]:
from pytket.extensions.braket import braket_to_tk
ghz4_3 = braket_to_tk(aws_ghz4_2)
render_circuit_jupyter(ghz4_3)

弊社Quantinuumのご紹介
- Quantinuum ウェブサイト（ 英語 ）： https://www.quantinuum.com/
- Quantinuum K.K. ウェブサイト（ 日本語 ）： https://quantinuum.co.jp/
- 各種技術詳細（ 英語 ）： https://www.quantinuum.com/products
- ニュース（ 日本語 ）： https://quantinuum.co.jp/news/  
- Twitter（ 日本語 ）： https://twitter.com/quantinuum_jp
- Quantinuum K.K.主催の勉強会（ 日本語 ）： https://quantinuum.connpass.com/  
- 採用情報（ 英語 ）：https://www.quantinuum.com/careers
- TKET slack channel：[TKET slack channel](https://join.slack.com/t/tketusers/shared_invite/zt-22295t975-eAsoPg0i4ANHajPAe61VzQ)

<img src="./fig/slack-qr.png" width="250">
